In [1]:
import json
import awkward1 as ak
import numpy as np

In [2]:
bikeroutes_json = open("Bikeroutes.geojson").read()
bikeroutes_pyobj = json.loads(bikeroutes_json)

In [3]:
bikeroutes = ak.Record(bikeroutes_pyobj)

In [4]:
bikeroutes

<Record ... [-87.7, 42], [-87.7, 42]]]}}]} type='{"type": string, "crs": {"type"...'>

In [5]:
ak.type(bikeroutes)

{"type": string, "crs": {"type": string, "properties": {"name": string}}, "features": var * {"type": string, "properties": {"STREET": string, "TYPE": string, "BIKEROUTE": string, "F_STREET": string, "T_STREET": option[string]}, "geometry": {"type": string, "coordinates": var * var * var * float64}}}

In [6]:
bikeroutes["features", "geometry", "coordinates"]

<Array [[[[-87.8, 41.9], ... [-87.7, 42]]]] type='1061 * var * var * var * float64'>

In [7]:
bikeroutes.features.geometry.coordinates

<Array [[[[-87.8, 41.9], ... [-87.7, 42]]]] type='1061 * var * var * var * float64'>

In [9]:
ak.to_list(bikeroutes.features[751])

{'type': 'Feature',
 'properties': {'STREET': 'E 26TH ST',
  'TYPE': '1',
  'BIKEROUTE': 'EXISTING BIKE LANE',
  'F_STREET': 'S STATE ST',
  'T_STREET': 'S DR MARTIN LUTHER KING JR DR'},
 'geometry': {'type': 'MultiLineString',
  'coordinates': [[[-87.62685625163756, 41.84558714841179],
    [-87.62675996392576, 41.84558902593194],
    [-87.62637708895348, 41.845596494328554],
    [-87.62626461651281, 41.845598326696425],
    [-87.62618268489398, 41.84559966093136],
    [-87.6261438116618, 41.84560027230502],
    [-87.62613206507362, 41.845600474403334],
    [-87.6261027723024, 41.8456009526551],
    [-87.62579736038116, 41.84560626159298],
    [-87.62553890383363, 41.845610239979905],
    [-87.62532611036139, 41.845613593674],
    [-87.6247932635836, 41.84562202574476]],
   [[-87.62532611036139, 41.845613593674],
    [-87.6247932635836, 41.84562202574476]],
   [[-87.6247932635836, 41.84562202574476],
    [-87.62446484629727, 41.84562675013391],
    [-87.62444032614908, 41.8456270927620

In [10]:
longitude = bikeroutes.features.geometry.coordinates[..., 0]

In [11]:
longitude

<Array [[[-87.8, -87.8, ... -87.7, -87.7]]] type='1061 * var * var * float64'>

In [12]:
latitude = bikeroutes.features.geometry.coordinates[..., 1]

In [13]:
latitude

<Array [[[41.9, 41.9, 41.9, ... 42, 42, 42]]] type='1061 * var * var * float64'>

In [14]:
np.mean(longitude)

-87.67152377693318

In [16]:
ak.mean(latitude, axis=1)

<Array [[41.9, 41.9, 41.9, ... 42, 42, 42]] type='1061 * var * ?float64'>

In [17]:
km_east = (longitude - np.mean(longitude)) * 82.7
km_north = (latitude - np.mean(latitude)) * 111.1
km_east, km_north

(<Array [[[-9.68, -9.69, ... -3.58, -3.62]]] type='1061 * var * var * float64'>,
 <Array [[[6.68, 6.68, 6.67, ... 9.68, 9.72]]] type='1061 * var * var * float64'>)

In [18]:
path = np.array([1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9])
path

array([1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9])

In [19]:
path[1:] - path[:-1]

array([1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1])

In [22]:
km_east[:, :, :-1] - km_east[:, :, 1:]

<Array [[[0.00603, 0.0165, ... 0.0385]]] type='1061 * var * var * float64'>

In [23]:
segment_length = np.sqrt((km_east[:, :, 1:] - km_east[:, :, :-1])**2 +
                         (km_north[:, :, 1:] - km_north[:, :, :-1])**2)
segment_length

<Array [[[0.00603, 0.0165, ... 0.0523]]] type='1061 * var * var * float64'>

In [25]:
ak.to_list(segment_length[751])

[[0.007965725361805015,
  0.03167462986536074,
  0.009303698353632218,
  0.006777366141816824,
  0.0032155337758260757,
  0.0009717022897204878,
  0.0024230948099507807,
  0.025264451818903598,
  0.021378926022712963,
  0.01760196411492381,
  0.0440763850916575],
 [0.0440763850916575],
 [0.027165180856806467, 0.0020281735093618165],
 [0.027165180856806467, 0.0020281735093618165],
 [0.02214495567782706],
 [0.007693515757648775,
  0.03667525065033862,
  0.02020647703022365,
  0.10374066852962399,
  0.13701231191288935,
  0.012466958457612119,
  0.12129772855897196,
  0.08473055433052853,
  0.012759495625968342,
  0.0007293106274618985],
 [0.007693515757648775, 0.03667525065033862]]

In [26]:
np.sum(segment_length[751], axis=-1)

<Array [0.171, 0.0441, ... 0.537, 0.0444] type='7 * float64'>

In [27]:
path_length = np.sum(segment_length, axis=-1)
path_length

<Array [[0.241], [0.0971], ... 0.347], [0.281]] type='1061 * var * float64'>

In [28]:
route_length = np.sum(path_length, axis=-1)
route_length

<Array [0.241, 0.0971, 0.203, ... 0.347, 0.281] type='1061 * float64'>

In [29]:
%%timeit

route_length = []
for route in bikeroutes_pyobj["features"]:
    path_length = []
    for polyline in route["geometry"]["coordinates"]:
        segment_length = []
        last = None
        for lng, lat in polyline:
            km_east = lng * 82.7
            km_north = lat * 111.1
            if last is not None:
                dx2 = (km_east - last[0])**2
                dy2 = (km_north - last[1])**2
                segment_length.append(np.sqrt(dx2 + dy2))
            last = (km_east, km_north)

        path_length.append(sum(segment_length))
    route_length.append(sum(path_length))

56.9 ms ± 2.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
%%timeit

km_east = bikeroutes.features.geometry.coordinates[..., 0] * 82.7
km_north = bikeroutes.features.geometry.coordinates[..., 1] * 111.1

segment_length = np.sqrt((km_east[:, :, 1:] - km_east[:, :, :-1])**2 +
                         (km_north[:, :, 1:] - km_north[:, :, :-1])**2)

path_length = np.sum(segment_length, axis=-1)
route_length = np.sum(path_length, axis=-1)

10.2 ms ± 196 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


![](bikeroutes-scaling.svg)